In [33]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split, cross_val_predict,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import metrics
import datetime

In [34]:
lemonade_sales = pandas.read_csv('./Lemonade.csv')
lemonade_sales.head()

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
0,1/1/2017,Sunday,27.0,2.00,15,0.3,10
1,2/1/2017,Monday,28.9,1.33,15,0.3,13
2,3/1/2017,Tuesday,34.5,1.33,27,0.3,15
3,4/1/2017,Wednesday,44.1,1.05,28,0.3,17
4,5/1/2017,Thursday,42.4,1.00,33,0.3,18


In [35]:
lemonade_sales.describe()
# sales = (temp * m1) + (rainfall*m2) + (flyers*m3) + (price*m4) + (iswednesday*m5) + (istuesday * m6)

,Temperature,Rainfall,Flyers,Price,Sales
count,365.000000,365.000000,365.000000,365.000000,365.000000
mean,60.731233,0.826603,40.284932,0.333973,25.323288
std,16.196266,0.273171,13.178651,0.075206,6.893589
min,15.100000,0.470000,9.000000,0.300000,7.000000
25%,49.700000,0.650000,31.000000,0.300000,20.000000
50%,61.100000,0.740000,39.000000,0.300000,25.000000
75%,71.300000,0.910000,49.000000,0.300000,30.000000
max,102.900000,2.500000,80.000000,0.500000,43.000000


In [36]:
dependent_variable = 'Sales'
independent_variables = ['Date','Day', 'Temperature', 'Rainfall', 'Flyers', 'Price']


In [37]:
for independent_variable in independent_variables:
#     print(lemonade_sales[independent_variable])
#     print(isinstance(lemonade_sales[independent_variable], list))
    print(isinstance(lemonade_sales[independent_variable][0], str))

True
True
False
False
False
False


In [38]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [39]:
################### HASTA ACA FUNCIONA LA REGRESION ######################### 

In [42]:
lemonade_sales['Date'] = pd.to_datetime(lemonade_sales['Date'])
y = lemonade_sales.iloc[:,-1]
X = lemonade_sales.iloc[:,1:-1]
# X = pd.get_dummies(data=X, prefix=['Day'])

In [43]:
X

,Day,Temperature,Rainfall,Flyers,Price
0,Sunday,27.0,2.00,15,0.3
1,Monday,28.9,1.33,15,0.3
2,Tuesday,34.5,1.33,27,0.3
3,Wednesday,44.1,1.05,28,0.3
4,Thursday,42.4,1.00,33,0.3
5,Friday,25.3,1.54,23,0.3
6,Saturday,32.9,1.54,19,0.3
7,Sunday,37.5,1.18,28,0.3
8,Monday,38.1,1.18,20,0.3
9,Tuesday,43.4,1.05,33,0.3


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print( X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(255, 5) (255,)
(110, 5) (110,)


In [101]:
linear_regression = LinearRegression().fit(X_train,y_train)
y_prediction_lr = linear_regression.predict(X_test)

In [139]:
svr = SVR(kernel='linear').fit(X_train,y_train)
y_prediction_svr = svr.predict(X_test)

In [140]:
cross_val_score(estimator=svr,X=X, y=y, cv=10)

array([0.89137375, 0.84410969, 0.64100821, 0.70718569, 0.81656048,
       0.90865598, 0.79010568, 0.54391941, 0.82170953, 0.86160397])

In [141]:
print(metrics.mean_absolute_error(y_test, y_prediction_lr))
print(metrics.mean_squared_error(y_test, y_prediction_lr))
print(numpy.sqrt(metrics.mean_squared_error(y_test, y_prediction_lr)))

0.8069253268858821
0.9726257237545004
0.9862178885796488


In [142]:
print(metrics.mean_absolute_error(y_test, y_prediction_svr))
print(metrics.mean_squared_error(y_test, y_prediction_svr))
print(numpy.sqrt(metrics.mean_squared_error(y_test, y_prediction_svr)))

0.8380673284531768
1.0111702452647413
1.005569612341553


In [112]:
for coefficient in linear_regression.coef_:
    print(coefficient)
#     print(X_test.columns[index]+': '+coefficient)

0.3565584047149493
-3.01319571699899
0.01973625265397505
2.2919369552726536
-0.17732649163008446
0.1394787806311346
-0.031070990925111196
-0.14215343236141348
0.17488712074497048
0.19418733591676823
-0.1580023223762647


In [115]:
coefficients = pandas.concat([pandas.DataFrame(X.columns),pandas.DataFrame(numpy.transpose(linear_regression.coef_)), columns], axis = 1)
coefficients

,0,0
0,Temperature,0.356558
1,Rainfall,-3.013196
2,Flyers,0.019736
3,Price,2.291937
4,Day_Friday,-0.177326
5,Day_Monday,0.139479
6,Day_Saturday,-0.031071
7,Day_Sunday,-0.142153
8,Day_Thursday,0.174887
9,Day_Tuesday,0.194187


In [120]:
linear_regression.intercept_

4.653180624951631

In [130]:
( 27   *   0.35 )   +   ( 2   *   ( 0   -   3 ) )   +   ( 15   *   0.01 )   +   ( 0.7   *   2.29 )   +   (0.194187) + linear_regression.intercept_

10.05036762495163